# Message Builder System Demonstration

This notebook demonstrates the **Message Builder System** - a fluent interface for constructing AWS Bedrock Converse API messages with automatic format detection and validation.

## Key Features

- 🏗️ **Fluent Interface**: Chain methods together for intuitive message construction
- 🔍 **Auto-Detection**: Automatic file format detection using magic bytes and extensions
- 📏 **Size Validation**: Built-in content size limits and validation
- 🎯 **Multi-Modal Support**: Text, images, documents, and videos in a single message
- 🛡️ **Error Handling**: Comprehensive validation and error reporting
- 🔄 **LLMManager Integration**: Seamless integration with the LLMManager system

## Setup and Imports

In [ ]:
import sys
import json
from pathlib import Path
import logging
from datetime import datetime
from typing import Dict, Any, List

# Add the src directory to path for imports
sys.path.append(str(Path.cwd().parent / "src"))

try:
    # Import Message Builder components
    from bedrock.models.message_builder_factory import (
        create_message, create_user_message, create_assistant_message
    )
    from bedrock.models.converse_message_builder import ConverseMessageBuilder
    from bedrock.models.message_builder_enums import (
        RolesEnum, ImageFormatEnum, DocumentFormatEnum, VideoFormatEnum
    )
    from bedrock.models.message_builder_constants import (
        MessageBuilderConfig, SupportedFormats
    )
    from bedrock.models.file_type_detector.file_type_detector import FileTypeDetector
    from bedrock.exceptions.llm_manager_exceptions import RequestValidationError
    
    print("✅ Message Builder imports successful!")
    IMPORTS_SUCCESSFUL = True
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("\n💡 Please ensure you're running this notebook from the notebooks directory")
    print("   and that the src directory contains the Message Builder modules.")
    IMPORTS_SUCCESSFUL = False

# Configure logging for better visibility
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print(f"📁 Working directory: {Path.cwd()}")
print(f"🔧 Message Builder System: {'Loaded' if IMPORTS_SUCCESSFUL else 'Failed to load'}")

## Helper Functions

In [ ]:
def read_file_bytes(file_path: str) -> bytes:
    """Read a file as raw bytes."""
    path = Path(file_path)
    if not path.exists():
        raise FileNotFoundError(f"File not found: {file_path}")
    
    with open(file_path, "rb") as file:
        return file.read()

def display_message_structure(message: Dict[str, Any], title: str = "Message Structure") -> None:
    """Display a formatted view of the message structure."""
    print(f"\n{title}")
    print("=" * len(title))
    
    print(f"🎭 Role: {message.get('role', 'unknown')}")
    
    content_blocks = message.get('content', [])
    print(f"📦 Content Blocks: {len(content_blocks)}")
    
    for i, block in enumerate(content_blocks, 1):
        print(f"\n   Block {i}:")
        
        if 'text' in block:
            text_preview = block['text'][:100] + "..." if len(block['text']) > 100 else block['text']
            print(f"     📝 Text: {text_preview}")
            print(f"     📏 Length: {len(block['text'])} characters")
        
        elif 'image' in block:
            image_info = block['image']
            format_type = image_info.get('format', 'unknown')
            size = len(image_info.get('source', {}).get('bytes', b''))
            print(f"     🖼️  Image: {format_type.upper()}")
            print(f"     📏 Size: {size:,} bytes ({size/1024:.1f} KB)")
        
        elif 'document' in block:
            doc_info = block['document']
            format_type = doc_info.get('format', 'unknown')
            name = doc_info.get('name', 'unnamed')
            size = len(doc_info.get('source', {}).get('bytes', b''))
            print(f"     📄 Document: {format_type.upper()}")
            print(f"     📝 Name: {name}")
            print(f"     📏 Size: {size:,} bytes ({size/1024:.1f} KB)")
        
        elif 'video' in block:
            video_info = block['video']
            format_type = video_info.get('format', 'unknown')
            size = len(video_info.get('source', {}).get('bytes', b''))
            print(f"     🎥 Video: {format_type.upper()}")
            print(f"     📏 Size: {size:,} bytes ({size/1024/1024:.1f} MB)")

print("✅ Helper functions defined!")

## Example 1: Basic Text Message Construction 💬

In [ ]:
if not IMPORTS_SUCCESSFUL:
    print("❌ Cannot run examples - Message Builder imports failed")
else:
    print("💬 Example 1: Basic Text Message Construction")
    print("=" * 45)

    # Traditional vs Message Builder construction
    traditional_message = {
        "role": "user",
        "content": [
            {
                "text": "Hello! Please introduce yourself and explain what you can help me with."
            }
        ]
    }

    # Message Builder construction
    builder_message = create_user_message() \
        .add_text("Hello! Please introduce yourself and explain what you can help me with.") \
        .build()
    
    display_message_structure(builder_message, "🏗️ Message Builder Result")
    
    print(f"\n✅ Messages are identical: {traditional_message == builder_message}")
    print("\n🏭 Different Factory Methods:")
    
    user_msg = create_user_message().add_text("User message").build()
    assistant_msg = create_assistant_message().add_text("Assistant message").build()
    
    print(f"   👤 User role: {user_msg['role']}")
    print(f"   🤖 Assistant role: {assistant_msg['role']}")

## Example 2: Image Message Construction 🖼️

In [ ]:
if not IMPORTS_SUCCESSFUL:
    print("❌ Cannot run image examples - Message Builder imports failed")
else:
    print("🖼️ Example 2: Image Message Construction")
    print("=" * 39)

    eiffel_image_path = Path("../images/1200px-Tour_Eiffel_Wikimedia_Commons_(cropped).jpg")
    if eiffel_image_path.exists():
        try:
            image_bytes = read_file_bytes(str(eiffel_image_path))
            print(f"📦 Image size: {len(image_bytes) / 1024:.1f} KB")
            
            # Message Builder with auto-detection
            image_message = create_user_message() \
                .add_text("Please analyze this image. What landmark do you see?") \
                .add_image_bytes(bytes=image_bytes, filename=eiffel_image_path.name) \
                .build()
            
            display_message_structure(image_message, "🏗️ Image Message Result")
            
            detected_format = image_message['content'][1]['image']['format']
            print(f"\n🔍 Auto-detected format: {detected_format.upper()}")
            
        except Exception as e:
            print(f"❌ Error: {e}")
    else:
        print(f"❌ Image file not found: {eiffel_image_path}")

## Example 3: Multi-Modal Message Construction 🎯

In [ ]:
if not IMPORTS_SUCCESSFUL:
    print("❌ Cannot run multi-modal examples - Message Builder imports failed")
else:
    print("🎯 Example 3: Multi-Modal Message Construction")
    print("=" * 45)

    # Create sample document content
    expert_content = """# Eiffel Tower Guide

## History
Built 1887-1889 for the World's Fair.
Designed by Gustave Eiffel.

## Details
- Height: 330 meters
- Material: Wrought iron
- Initially controversial
"""
    
    doc_bytes = expert_content.encode('utf-8')
    print(f"📄 Created document: {len(doc_bytes)} bytes")
    
    try:
        # Complex multi-modal message with fluent interface
        multi_modal_message = create_user_message() \
            .add_text("You are a professional tour guide. Here's reference material:") \
            .add_document_bytes(
                bytes=doc_bytes, 
                format=DocumentFormatEnum.MD,
                name="Eiffel Tower Reference"
            ) \
            .add_text("Now analyze this image using the reference information:") \
            .build()
        
        # Try to add image if available
        eiffel_path = Path("../images/1200px-Tour_Eiffel_Wikimedia_Commons_(cropped).jpg")
        if eiffel_path.exists():
            image_bytes = read_file_bytes(str(eiffel_path))
            
            # Add image to existing message structure
            enhanced_message = create_user_message() \
                .add_text("You are a professional tour guide. Here's reference material:") \
                .add_document_bytes(
                    bytes=doc_bytes, 
                    format=DocumentFormatEnum.MD,
                    name="Eiffel Tower Reference"
                ) \
                .add_text("Now analyze this image using the reference information:") \
                .add_image_bytes(bytes=image_bytes, filename=eiffel_path.name) \
                .add_text("Provide expert analysis combining the reference and visual information.") \
                .build()
            
            display_message_structure(enhanced_message, "🎯 Complete Multi-Modal Message")
            
            print(f"\n📊 Message Statistics:")
            print(f"   📦 Total blocks: {len(enhanced_message['content'])}")
            print(f"   📝 Text blocks: {sum(1 for block in enhanced_message['content'] if 'text' in block)}")
            print(f"   📄 Document blocks: {sum(1 for block in enhanced_message['content'] if 'document' in block)}")
            print(f"   🖼️ Image blocks: {sum(1 for block in enhanced_message['content'] if 'image' in block)}")
            
        else:
            display_message_structure(multi_modal_message, "🎯 Document-Based Message")
            
    except Exception as e:
        print(f"❌ Error in multi-modal construction: {e}")

## Summary and Benefits 📋

In [ ]:
print("📋 Message Builder System Summary")
print("=" * 35)

if IMPORTS_SUCCESSFUL:
    print("\n✅ Capabilities Demonstrated:")
    print("   💬 Text Messages - Clean fluent interface")
    print("   🖼️ Image Messages - Automatic format detection")
    print("   📄 Document Messages - Built-in validation")
    print("   🎯 Multi-Modal - Complex messages made simple")
    print("   🏗️ Factory Functions - Easy role-based creation")

    print("\n🌟 Key Benefits:")
    print("   ✅ Fluent interface - Readable, chainable methods")
    print("   ✅ Auto-detection - Format detection from filenames/content")
    print("   ✅ Type safety - Strong typing with enums")
    print("   ✅ Validation - Size limits and error handling")
    print("   ✅ Integration - Seamless LLMManager compatibility")

    print("\n🚀 Next Steps:")
    print("   1. Use Message Builder in your applications")
    print("   2. Integrate with LLMManager for API calls")
    print("   3. Explore video and advanced features")
    print("   4. Build complex multi-modal AI applications")

    print("\n🎉 Message Builder makes multi-modal AI development elegant!")
else:
    print("\n❌ Import issues prevented demonstration")
    print("\n💡 To fix: Ensure src directory contains Message Builder modules")
    
print("\n📊 Demonstration Complete!")